In [ ]:
#Block 0: Documentation

print('Program to process and visualize ABI AOD data, Maine DEP Aerosol and Trace Gases Satellite Product Training, November 10, 2020\n')
print('Version 1.0, October 14, 2020\n')
print('Written by Dr. Amy Huff (IMSG at NOAA/NESDIS/STAR) and Ryan Theurer (GVT LLC at NOAA/NESDIS/STAR)\n')
print('For questions contact Dr. Huff: amy.huff@noaa.gov\n')
print('This program processes GOES-16 ABI aerosol optical depth (AOD) using data quality flags (DQF) and plots processed AOD on a map to create a professional-looking figure.\n')
print('Code block 1 imports settings and libraries, and code blocks 2-8 are functions that require no input from the user; there is no visible output from these blocks. Code block 9 is where we adjust settings and obtain output.')

In [ ]:
#Block 1: Import libraries and settings

#To perform array operations
import numpy as np

#Main plotting libraries
import matplotlib as mpl
from matplotlib import pyplot as plt

#To create maps
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#To access files in the directory
import os

#To read in netCDF files
from netCDF4 import Dataset

#Module for manipulating dates and times
import datetime

#To collect lists of files from folders
import glob

import warnings
warnings.filterwarnings('ignore')

#Sets default font size to 12
plt.rcParams.update({'font.size': 12})

#Option to keep numpy from printing in scientific notation by default
np.set_printoptions(suppress = True)

In [ ]:
#Block 2: Algorithm to convert latitude and longitude radian values to degrees

def degrees(file_id):
    proj_info = file_id.variables['goes_imager_projection']
    lon_origin = proj_info.longitude_of_projection_origin
    H = proj_info.perspective_point_height+proj_info.semi_major_axis
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis
    
    #Data info
    lat_rad_1d = file_id.variables['x'][:]
    lon_rad_1d = file_id.variables['y'][:]
    
    #Create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)
    
    #lat/lon calculus routine from satellite radian angle vectors
    lambda_0 = (lon_origin*np.pi)/180.0
    
    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*(np.power(np.cos(lon_rad),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)
    
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    
    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)
    
    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return lat, lon

In [ ]:
#Block 3: Select and process AOD data using DQF

def aod_data(file_id):
    #Read in AOD data
    aod_data = file_id.variables['AOD'][:,:]

    #Select quality of AOD data by masking pixels we don't want to display using DQF variable
    #high quality: DQF = 0, medium quality: DQF = 1, low quality: DQF = 2, not retrieved (NR): DQF = 3
    #Use high + medium qualities ("top 2 qualities") for operational applications (e.g., mask low quality and NR pixels)
    dqf = file_id.variables['DQF'][:,:]
    if quality == 'high':
        quality_mask = (dqf != 0)
    if quality == 'top2':
        quality_mask = (dqf > 1)
    if quality == 'all':
        quality_mask = (dqf > 2)
    aod = np.ma.masked_where(quality_mask, aod_data) 
    
    return aod

In [ ]:
#Block 4: Create AOD color bar, indepenent of plotted data, auto-positioning
#cbar_ax are dummy variables
#Location/dimensions of colorbar set by .set_position (x0, y0, width, height) to scale automatically with figure
#"bar" (color map) and "max_color" (color for AOD > 1) are variables we will set in Block 9

def aod_colorbar(fig, ax):
    last_axes = plt.gca()
    cbar_ax = fig.add_axes([0, 0, 0, 0])
    plt.draw()
    posn = ax.get_position()
    cbar_ax.set_position([0.35, posn.y0 - 0.06, 0.3, 0.02])
    norm = mpl.colors.Normalize(vmin = 0, vmax = 1)
    color_map = plt.get_cmap(bar)
    color_map.set_over(max_color)
    cb = mpl.colorbar.ColorbarBase(cbar_ax, cmap = color_map, norm = norm, orientation = 'horizontal', ticks = [0, 0.25, 0.5, 0.75, 1], extend = 'max')
    cb.set_label(label = 'Aerosol Optical Depth', size = 10, weight = 'bold')
    cb.ax.set_xticklabels(['0', '0.25', '0.50', '0.75', '1.0'])
    cb.ax.tick_params(labelsize = 10)
    plt.sca(last_axes)

In [ ]:
#Block 5: Set range for plotting AOD data
# The numpy "arange(data min, data max, contour interval)" command sets the data range for AOD data
#"contour_interval" is a variable we will set in Block 9

def aod_data_range():    
    aod_data_range = np.arange(0, 1.1, contour_interval)
    
    return aod_data_range

In [ ]:
#Block 6: Format map with lat/lon grid, coastlines/borders, and set map domain

def map_settings(ax):
    #Set up and label the lat/lon grid
    lon_formatter = LongitudeFormatter()
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.set_xticks(lon_ticks, crs = ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs = ccrs.PlateCarree())
    
    #Set lat/lon ticks and gridlines
    ax.tick_params(length = 5, direction = 'out', labelsize = 10, pad = 5)
    ax.grid(linewidth = 0.5, zorder = 3)
   
    #Draw coastlines/borders using Cartopy; zorder sets drawing order for layers
    ax.coastlines(resolution = '50m', zorder = 3)
    ax.add_feature(cfeature.NaturalEarthFeature(category = 'physical', name = 'ocean', scale = '50m'), facecolor = 'lightgrey')
    ax.add_feature(cfeature.NaturalEarthFeature(category = 'physical', name = 'land', scale = '50m'), facecolor = 'grey')
    ax.add_feature(cfeature.NaturalEarthFeature(category = 'physical', name = 'lakes', scale = '50m'), facecolor = 'lightgrey', edgecolor = 'black', zorder = 2)
    ax.add_feature(cfeature.NaturalEarthFeature(category = 'cultural', name = 'admin_0_countries', scale = '50m'), facecolor = 'none', lw = 0.5, edgecolor = 'black', zorder = 3)
    ax.add_feature(cfeature.NaturalEarthFeature(category = 'cultural', name = 'admin_1_states_provinces', scale = '50m'), facecolor = 'none', lw = 0.5, edgecolor = 'black', zorder = 3)
    
    #Set lat/lon extents for map [x0, x1, y0, y1]
    #"domain" is a variable we will set in Block 9
    ax.set_extent(domain, crs = ccrs.PlateCarree())

In [ ]:
#Block 7: Create plot title from ABI data file name

def abi_plot_title(fname):
    #Pull Julian date from file name, convert to Gregorian date, and reformat
    julian = datetime.datetime.strptime(fname[-49:-42], '%Y%j').date()
    date = julian.strftime('%d %b %Y')
    
    #Set AOD quality level
    if quality == 'high':
        quality_title = '(High Quality)'
    if quality == 'top2':
        quality_title = '(High and Medium Quality)'
    if quality == 'all':
        quality_title = '(High, Medium, and Low Quality)'

    #Create plot title
    abi_title = 'GOES-'+ fname[-53:-51] + '/ABI\n' + 'Aerosol Optical Depth ' + quality_title + '\n' + fname[-42:-40] + ':' + fname[-40:-38] + ' UTC, ' + date
        
    return abi_title

In [ ]:
#Block 8: Plot ABI AOD data on Plate Carree map projection

def plot_abi_aod(file_id):

    #Set up figure and map projection: PlateCarree(central_longitude)
    #Plate Carree: equidistant cylindrical projection w/equator as the standard parallel; default central_longitude = 0
    fig = plt.figure(figsize=(8, 10))
    ax = fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())

    #Select and process ABI AOD data
    aod = aod_data(file_id)

    #Read in ABI latitutude and longitude values in degrees
    lat, lon = degrees(file_id)

    #Format map
    map_settings(ax)

    #Add title
    plt.title(abi_plot_title(fname), pad = 10, ma = 'center', size = 12, weight = 'bold')
    
    #Add AOD color bar
    aod_colorbar(fig, ax)

    #Plotting settings for AOD data
    data_range = aod_data_range()
    color_map = plt.get_cmap(bar)
    color_map.set_over(max_color)

    #Create filled contour plot of AOD data
    ax.contourf(lon, lat, aod, data_range, cmap = color_map, extend = 'both', zorder = 3, transform = ccrs.PlateCarree())

    #Show figure
    plt.show()

    #Save figure as a .png file
    date = datetime.datetime.strptime(fname[-49:-42], '%Y%j').date().strftime('%Y%m%d')
    filename = 'G' + fname[-53:-51] + '_ABI_AOD_' + quality + '_' + date + '_' + fname[-42:-38]
    fig.savefig(filename, bbox_inches = 'tight', dpi = file_res)
    
    #Close file
    file_id.close()
    
    #Erase plot so we can build the next one
    plt.close()

In [ ]:
#Block 9: Create figure(s) of ABI AOD data

#File settings
file_path = os.getcwd()  #location where ABI data files are stored and where figures will be saved
file_name = '/OR_ABI-L2-AODC-M6_G16_s20202601501123_e20202601503496_c20202601506349.nc'  #for plotting a single file
figures = 'single'  #number of figures to create: 'single' (one data file) or 'multiple' (multiple data files)

#Data settings
quality = 'top2'  #AOD data quality: 'high', 'top2', or 'all'
contour_interval = 0.05  #AOD contour interval: 0.1 = runs faster/coarser resolution, 0.01 = runs slower/higher resolution

#Plot settings
bar = 'rainbow'  #color map for colorbar and plot
max_color = 'darkred' #color for AOD data > 1
file_res = 150  #dpi setting for image

#Mapping settings
#Use 180 degrees for longitude (i.e, 100 degrees W = -100)
domain = [-125, -65, 20, 50]  #[x0, x1, y0, y1]
lon_ticks = [-125, -105, -85, -65]
lat_ticks = [20, 30, 40, 50]


if figures == 'single':
    #Open single data file and set file name
    fname = file_path + file_name
    file_id = Dataset(fname)
    plot_abi_aod(file_id)
    print('Done!')

if figures == 'multiple':
    #Collect all of the .nc files in the given subdirectory
    file_list = sorted(glob.glob(file_path + '/*.nc')) 
    #Loop through data files, making/saving a figure for each data file
    for fname in file_list:
        file_id = Dataset(fname)
        plot_abi_aod(file_id)
    print('Done!')